# Session 12 - Spatial analysis, hierarchy manipulation, and cell interactions
*Goal: Learn a variety of tools within QuPath to analyze the spatial relationships between cells*

## Create Tingible Body Macrophage Annotations
When analyzing spatial relationships, *detections* are characterized by their centroids while *annotations* are characterized by their boundaries. To determine if macrophages are plausibly touching T cells, we need to measure distance from the macrophage boundaries. Therefore, we need to create annotations around the macrophage. There are two ways to do this:

1. Convert detections to annotations with a script
    - Pros: computationally fast, utilizes object classifiers, finds TB Macs in the entire tissue <br>
2. Create new annotation objects
    - Pros: no scripting, limits potential area to germinal centers
    
We'll go through both in the workshop (time permitting)

### Option 1- Turn detections into annotations with a script
The script below takes the all of the detection objects currently classified as TBMacs and creates annotations with the exact same shape. It then deletes the detection objects. 

Copy paste the script into the Script editor window and run it (`Run > Run`). If you named your macrophages anything other than 'TBMacs' (case sensitive), change line 2 to have your class name. 

In [ ]:
def macs = getDetectionObjects() //find all detection objects
def tbmacs = macs.findAll{it.classifications.contains('TBMacs')} //filter for the tingible body class
def newAnnots = tbmacs.collect{ //loop through each object individually, collecting the results into newAnnots
    //"it" refers to each cell as it goes through the loop
    PathObjects.createAnnotationObject(it.getROI(),it.getPathClass(),it.getMeasurementList()) // create an object with the same region of interest, classification, and measurements
} 

addObjects(newAnnots) //add the new annotations to the image, otherwise they are just saved in the variable and not real
removeObjects(macs,true) //delete all detections

### Option 2- Create annotations directly from the pixel classifier
Since tingible body macrophages are predominantly found in the germinal center in tonsils, and they are the dominant CD11c+ CD68+ cells in the germinal centers,  we can skip the entire object classifier step and directly segment GC cells found by our pixel classifier as TB Mac annotations.

For this workshop, I recommend Option 2. 

**In a real project, which method you should use depends on the underlying biology** There are CD11c+CD68+ cells outside the germinal center. If these are the cells of interest, than you want to use Option 1, which finds cells regardless of location. If these are definitionally not the cells we care about, then restricting the region to GCs improves accuracy and makes us less likely to miss small cells. 

1. Delete all the detections (`Objects > Delete > Delete all detections`)
2. Create a new class called 'GC'
2. Use the annotation tools to mark the germinal centers in the tissue section. Give them the GC class. 
    <img src="Images/GCs.PNG">
    
3. Select all of the germinal centers
4. Load the pixel classifier 
    1. `Classify > Pixel classification > Load pixel classifier`
    2. Choose model "TBMacs"
    3. <kbd>Create objects</kbd>
    4. Choose parent objects: "Current selection"
    5. Same options as previously, except New object type = "Annotations" <br>
        <img src="Images/MacAnnots.PNG">
    6. Click <kbd>OK</kbd>

## Detect non-macrophage cells
We are about to create cells in the full tissue, but we don't want it to erase the germinal center and macrophage annotations. So, we'll need to alter the hierarchy before cell creation. 

1. Delete the full image rectangle. **KEEP** child objects!
2. Create a new full image rectangle. Do NOT re-resolve the hieararchy. We specifically want the germinal centers to be outside of this rectangle. 

1. Select the full image rectangle
2. Run InstanSeg: `Extensions > InstanSeg > Run InstanSeg`
    - Pick whichever channels you feel are appropriate
    - I don't recommend running measurements
4. If you cannot detect the cells in a reasonable amount of time, download the project "Proj2 - Raw InstanSeg" and proceed from here!
4. Find internal cells
    1. `Analyze > Spatial Analysis > Signed distance to annotations 2D`
    2. Click <kbd>Yes</kbd> when it asks to split multipart classifications?
    3. This will take a minute.
5. Look at the measurements for each cell. The ones inside the macrophages have a negative value for 'Signed distance to annotation with TBMacs µm'. 
6. Remove the internal cells
    1. `Classify > Object classification > Create single measurement classifier`
    2. Set the measurement to 'Signed distance to annotation with TBMacs µm'
    3. Set the threshold to 0
    4. Set the Below class to anything not already claimed. I used "Other"
    5. Give the classifier a name, <kbd>Save</kbd>, and <kbd>Apply</kbd>
    
        <img src="Images/InsideMacClass.PNG">
7. Select all Other cells (or whatever class you used) and delete them. 

## Quantify cell-cell interactions

1. Train a classifier for PD-1 
    1. Select all cells
    2. Measure the mean, min, max, standard deviation, and Haralick features of the PD-1 and Ki67 channels
    3. Select the GC objects and lock them. (`Right click > Annotations > Lock`)
    4. Use the points annotation tool to [train an object classifier](./Session%206%20-%20Classifying%20Cells%20pt1.ipynb#3.-Finding-Leukocytes---Training-an-ML-Object-Classifier) for PD-1+ cells. Use only PD1 channel measurements. 

2. Train a classifier for Ki67
    1. Lock the two point annotation objects. Locked annotations are not used when training a classifier
    2. Rename the Ignore* Points "PD1 Ignore"
    3. Train an object classifier to find Ki67+ cells using only the Ki67 measurments. Make sure to create a new Ignore* points object for this classifier, named "Ki67 Ignore". 
    
3. Create a classifier to identify cells very near macrophages.
    1. `Classify > Object classification > Create single measurement classifier`
    2. Set the measurement to 'Signed distance to annotation with TBMacs µm'
    3. Set the threshold to 5 (remember, you've already deleted cells with distance <0)
    4. Creating a "touching" class. Set the Below class to "touching"
    5. Give the classifier a name, <kbd>Save</kbd>, and <kbd>Apply</kbd>
    
3. Combine the 3 classifiers. 
4. Save the file. Duplicate the image, with a relevant name. Delete the training points. 

5. [Export](./Session%209%20-%20Data%20Export%20and%20Simplification.ipynb#Exporting-Cell-Counts) the annotation measurements
    1. `Measure > Export measurements`
    2. Use these settings
        <img src="Images/GCCellCount.PNG">


### Are the cells near Tingible Body Macrophages more likely to be PD1+ than the ones further away?
open 

